In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### What is Gradient Descent ?
+ Gradient descent is an optimization algorithm used to minimize the cost function $J(w, b)$ by updating the parameters $w$ and $b$ in the direction that reduces the cost the most.
### Update rule
+ In each iteraction, gradient descent performs the follwing updates:
$$
\begin{aligned}
w = w - \alpha \frac{\partial J(w,b)}{\partial w} \\
b = b - \alpha \frac{\partial J(w,b)}{\partial b}
\end{aligned}
$$
+ The gradient is defined as:
$$
J(w, b) = \frac{1}{2m} \sum_{i = 0}^{m - 1}(f_{w, b}(X^{(i)}) - y^{(i)})^2
$$
$$
\Leftrightarrow J(w, b) = \frac{1}{2m} \sum_{i = 0}^{m - 1}(wx^{(i)} + b - y^{(i)})^2
$$
+ We want to compute:
$$
    \frac{\partial J(w, b)}{\partial w}
$$
$$
    \frac{\partial J(w, b)}{\partial w} = \frac{1}{2m} \sum_{i = 0}^{m - 1} 2(wx^{(i)} + b - y^{(i)})\cdot x^{(i)}
$$
+ We want to compute:
$$
    \frac{\partial J(w, b)}{\partial b}
$$
$$
    \frac{\partial J(w, b)}{\partial b} = \frac{1}{2m} \sum_{i = 0}^{m - 1} 2(wx^{(i)} + b - y^{(i)})\cdot 1
$$
    

